In [1]:
import math
import torch
from  torch import nn
import dltools

In [3]:
num_hiddens ,num_heads = 100,5
attention = dltools.MultiHeadAttention(num_hiddens,num_hiddens,num_hiddens,num_hiddens,num_heads,0.2
                                       )
attention.eval()

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

In [4]:
batch_size ,num_queries,valid_lens  =2,4,torch.tensor([3,2])
X = torch.ones((batch_size,num_queries,num_hiddens))
attention(X,X,X,valid_lens).shape

torch.Size([2, 4, 100])

In [5]:
class PositionalEncoding(nn.Module):
    def __init__(self, num_hiddens,dropout,max_len=1000,**kwargs):
        super().__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        self.P = torch.zeros((1,max_len,num_hiddens))
        X = torch.arange(max_len,dtype=torch.float32).reshape(-1,1) / torch.pow(10000,torch.arange(0,num_hiddens,2,dtype=torch.float32)/num_hiddens)
        self.P[:,:,0::2] = torch.sin(X)
        self.P[:,:,1::2] = torch.cos(X)
    def forward(self,X):
        X = X + self.P[:, :X.shape[1],:].to(X.device)
        return self.dropout(X)


In [6]:
encoding_dim,num_steps = 32,60
pos_encodin = PositionalEncoding(encoding_dim,0)
pos_encodin.eval()
X = pos_encodin(torch.zeros(1,num_steps,encoding_dim))
P = pos_encodin.P[:,:X.shape[1],:]
X.shape,P.shape

(torch.Size([1, 60, 32]), torch.Size([1, 60, 32]))